In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

# Preguntas

## Consistencia
Los datos podrian ser inconsistentes dentro de un mismo archivo, o entre archivos.

## Coherencia
Que rango de fechas cubre cada archivo? Hablan todos del mismo periodo?